In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/openai-community/gpt2/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)


INFO 07-16 02:56:51 [__init__.py:244] Automatically detected platform cuda.
INFO 07-16 02:57:05 [config.py:841] This model supports multiple tasks: {'reward', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.
ERROR 07-16 02:57:05 [config.py:130] Error retrieving safetensors: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/data/zju-46/shenyl/hf/model/openai-community/gpt2/'. Use `repo_type` argument if needed., retrying 1 of 2
ERROR 07-16 02:57:07 [config.py:128] Error retrieving safetensors: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/data/zju-46/shenyl/hf/model/openai-community/gpt2/'. Use `repo_type` argument if needed.
INFO 07-16 02:57:07 [config.py:3378] Downcasting torch.float32 to torch.bfloat16.
INFO 07-16 02:57:07 [config.py:1472] Using max model len 1024
WARNING 07-16 02:57:08 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
I

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-16 02:57:10 [default_loader.py:272] Loading weights took 0.24 seconds
INFO 07-16 02:57:10 [model_runner.py:1255] Model loading took 0.2378 GiB and 0.279352 seconds
INFO 07-16 02:57:11 [worker.py:295] Memory profiling takes 0.68 seconds
INFO 07-16 02:57:11 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-16 02:57:11 [worker.py:295] model weights take 0.24GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 41.93GiB.
INFO 07-16 02:57:12 [executor_base.py:115] # cuda blocks: 76329, # CPU blocks: 7281
INFO 07-16 02:57:12 [executor_base.py:120] Maximum concurrency for 1024 tokens per request: 1192.64x
INFO 07-16 02:57:16 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 5.29 seconds


In [2]:
# Create a simple example prompt for testing
example = "How to spend our life?"
# Generate baseline response without steering
example_answers = llm.generate(
    example,
    SamplingParams(
        temperature=0,
        max_tokens=50,
        skip_special_tokens=False,
    ),
)

# Display baseline response
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)

sv_request = SteerVectorRequest(
    steer_vector_name="lm_steer",
    steer_vector_id=1,
    steer_vector_local_path="gpt2.pt", # https://huggingface.co/spaces/Glaciohound/LM-Steer/blob/main/checkpoints/gpt2.pt
    prefill_trigger_positions=[-1],
    algorithm="lm_steer",
    scale=1.0, # here positive - Toxification, negative - Detoxification
    target_layers=[11] # last layer
)
# Generate response with lm_steer
output = llm.generate(
    example, 
    SamplingParams(
        temperature=0,
        max_tokens=50,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display lm_steer response
print("=====lm_steer=====")
print(output[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Baseline=====


The answer is simple: Spend your life.

The best way to spend your life is to spend it wisely.

The best way to spend your life is to spend it wisely.

The best way to spend your life


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====lm_steer=====
ONSORED

The answer is simple: We can't.

The answer is that we can't spend our life.

The answer is that we can't spend our life.

The answer is that we can't spend our life
